In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F 
import torch.utils.data
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

#Source: https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/

In [2]:
#Define hyper-parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
#Load the data
data_folder = './MNIST/data/'
train_filename = data_folder + 'train_images.pkl'
labels_filename = data_folder + 'train_labels.csv'
test_filename = data_folder + 'test_images.pkl'

train_images = pd.read_pickle(train_filename)
train_labels = pd.read_csv(labels_filename)
test_images = pd.read_pickle(test_filename)

train_labels = train_labels['Category'].values

X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.15)

In [4]:
#Transform data
#create feature and target tensor for train and test
torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

torch_X_train = torch_X_train.view(-1, 1,64,64).float()
torch_X_test = torch_X_test.view(-1,1,64,64).float()
#train.size = torch.Size([34000, 1, 64, 64])
#test.size = torch.Size([6000, 1, 64, 64])

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), #in, out
            # 32 is kernel number 
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #we now have 32*32 images
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #now its 16*16
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
            #now its 8*8
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(8 * 8 * 128, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


In [6]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/340], Loss: 2.4160, Accuracy: 21.00%
Epoch [1/5], Step [200/340], Loss: 2.2572, Accuracy: 16.00%
Epoch [1/5], Step [300/340], Loss: 2.1692, Accuracy: 18.00%
Epoch [2/5], Step [100/340], Loss: 2.0915, Accuracy: 25.00%
Epoch [2/5], Step [200/340], Loss: 2.0848, Accuracy: 20.00%
Epoch [2/5], Step [300/340], Loss: 1.9001, Accuracy: 28.00%
Epoch [3/5], Step [100/340], Loss: 1.7033, Accuracy: 39.00%
Epoch [3/5], Step [200/340], Loss: 1.2938, Accuracy: 55.00%
Epoch [3/5], Step [300/340], Loss: 1.4988, Accuracy: 44.00%
Epoch [4/5], Step [100/340], Loss: 1.2713, Accuracy: 55.00%
Epoch [4/5], Step [200/340], Loss: 0.8195, Accuracy: 74.00%
Epoch [4/5], Step [300/340], Loss: 1.2039, Accuracy: 64.00%
Epoch [5/5], Step [100/340], Loss: 0.9993, Accuracy: 68.00%
Epoch [5/5], Step [200/340], Loss: 0.5612, Accuracy: 83.00%
Epoch [5/5], Step [300/340], Loss: 0.8083, Accuracy: 80.00%


In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), 'deep_cnn_model.ckpt')

Test Accuracy of the model on the 10000 test images: 67.15 %


In [9]:
#save trained model
